## Parte 3: Construyendo un Sistema de Recomendacion con Feedback Implicito

En este ejercicio, desarrollaremos un sistema de recomendacion con feedback implicito utilizando la libreria     [implicit](https://github.com/benfred/implicit).

**Pero, a que nos referimos con feedback implicito?**

En el primer ejercicio abordamos el filtro colaborativo el cual se basa en la suposicion de que `usuarios similares gustan de las mismas cosas/items`. La matriz usuario-item, o "matriz de utilidad" es la piedra angular del filtrado colaborativo. En la matriz de utilidad las filas representan a los usuarios y las columnas representan a los items.



Las celdas de la matriz se llenan a partir del grado de preferencia de un usuario a un item determinado y esto se representa en cualquiera de las dos formas:
1. **Feedback explicito:** feedback directo hacia un item (por ejemplo el weighted_rating de una pelicula como lo vimos en el [Ejercicio 1](https://experiencia21.tec.mx/courses/481176/assignments/15386625?module_item_id=28379086))

2. **Feedback implicito:** comportamiento indirecto hacia un item (por ejemplo el historial de compra, el historial de navegacion o historial de busquedas)

El feedback implicito hace suposiciones sobre las preferencias del usuario a partir de las acciones hacia dichos items. Si retomamos el ejemplo si miraste todos los episodios de un show y viste todas las temporadas en una semana, entonces existe la elevada posibilidad de que te guste ese show. Sin embargo, si empiezas a mirar una serie y te detienes a la mitad del primer episodio, entonces es probable que se pueda asumir que no te haya gustado ese show.



### Paso 1: Agregando las Librerias

Estos seran las librerias que utilizaremos:

- [numpy](https://numpy.org/)
- [pandas](https://pandas.pydata.org/)
- [implicit](https://github.com/benfred/implicit)
- scipy (en especifico la clase **csr_matrix**)

In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix

import implicit

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### Paso 2: Cargando los datos

Dado que ya te has familiarizado con el dataset de itemLens de los ejercicios 1 y 2 en este ejercicio continuaremos utilizando este dataset que puede encontrar[aqui](https://grouplens.org/datasets/itemlens/), o lo puedes descargar directamente de [aqui](http://files.grouplens.org/datasets/itemlens/ml-latest-small.zip). (Recuerda que estamos trabajando con los datasets `ml-latest-small.zip` )


In [2]:
from sklearn.preprocessing import MinMaxScaler

pd.options.mode.copy_on_write = True
steam_games_df_base = pd.read_json('steam_games_df.json', lines=True)

reviews_df_base = pd.read_json('reviews_df.json', lines=True)

user_items_df_base = pd.read_json('user_items_df.json', lines=True)
steam_games_df = steam_games_df_base

reviews_df = reviews_df_base

user_items_df = user_items_df_base
# Preprocesamiento
#steam games
steam_games_df = steam_games_df[steam_games_df['item_id'].notna()]
steam_games_df.reset_index(drop=True, inplace=True)
steam_games_df['item_id'] = steam_games_df['item_id'].astype(int)
#steam_games_df
#reviews 
reviews_df = reviews_df[reviews_df['item_id'].notna()]
reviews_df.reset_index(drop=True, inplace=True)
reviews_df['item_id'] = reviews_df['item_id'].astype(int)
#Si no esta en nuestra base de steam games. 
reviews_df = reviews_df[reviews_df['item_id'].isin(steam_games_df['item_id'])]
reviews_df = reviews_df.drop(columns=['funny', 'posted', 'last_edited', 'helpful', 'user_url'], axis=1)
#reviews_df
# Este cambio se hizo antes -> steam_games_df.rename(columns={'id': 'item_id'}, inplace=True)
#User Items
user_items_df = user_items_df[user_items_df['item_id'].isin(steam_games_df['item_id'])]
user_items_df['item_id'] = user_items_df['item_id'].astype(int)
user_items_df = user_items_df.drop(columns=['user_url'], axis=1)
user_items_df = user_items_df[user_items_df['playtime_forever'] != 0.0]
#user_items_df


def sigmoid(x, scale=1):
    return 1 / (1 + np.exp(-x * scale))


user_items_df['log_ptime'] = np.log1p(user_items_df['playtime_forever'])
user_items_df['exp_ptime_2week'] = sigmoid(user_items_df['playtime_2weeks'], scale=0.3)
max_value = 10
user_items_df['exp_ptime_2week'] *= max_value
scaler_playtime = MinMaxScaler(feature_range=(0, 5))
user_items_df['log_ptime_scaled'] = scaler_playtime.fit_transform(user_items_df[['log_ptime']].values.reshape(-1, 1))

scaler_lastweek = MinMaxScaler(feature_range=(0, 5))
user_items_df['exp_ptime_2week_scaled'] = scaler_lastweek.fit_transform(
    user_items_df[['exp_ptime_2week']].values.reshape(-1, 1))
n = 0.6  # Peso para el tiempo de juego en las últimas dos semanas
m = 0.4  # Peso para el tiempo de juego total
user_items_df['combined_rating'] = n * user_items_df['exp_ptime_2week_scaled'] + m * user_items_df['log_ptime_scaled']
user_items_df = pd.merge(user_items_df, reviews_df[['user_id', 'item_id', 'recommend']], on=['user_id', 'item_id'],
                         how='left')
#result = pd.merge(user_items_df, reviews_df, on=['user_id', 'item_id'],how='left')

#user_items_df
#result
user_items_df['weighted_rating'] = user_items_df['combined_rating'] * (1 + user_items_df['recommend'].fillna(0))

Revisamos el contenido de user_items_df

In [3]:
user_items_df.head()

,item_id,item_name,playtime_forever,playtime_2weeks,user_id,log_ptime,exp_ptime_2week,log_ptime_scaled,exp_ptime_2week_scaled,combined_rating,recommend,weighted_rating
0,10,Counter-Strike,6.0,0.0,76561197970982479,1.945910,5.0,0.493976,0.0,0.197590,NaN,0.197590
1,30,Day of Defeat,7.0,0.0,76561197970982479,2.079442,5.0,0.546629,0.0,0.218651,NaN,0.218651
2,300,Day of Defeat: Source,4733.0,0.0,76561197970982479,8.462526,5.0,3.063538,0.0,1.225415,NaN,1.225415
3,240,Counter-Strike: Source,1853.0,0.0,76561197970982479,7.525101,5.0,2.693903,0.0,1.077561,NaN,1.077561
4,3830,Psychonauts,333.0,0.0,76561197970982479,5.811141,5.0,2.018072,0.0,0.807229,NaN,0.807229


En este ejercicio, definiremos el weighted_rating de las peliculas como el numero de veces que un usuario las ha mirado. Por ejemplo, si Jimena (una usuaria en nuestro dataset) le dio a la pelicula de`Batman` un weighted_rating de 1 y a `Jurassic Park` un weighted_rating de 5, podemos asumir que ha mirado la pelicula de Batman una vez y la de Jurassic Park un total de 5 veces.

### NOTA ###

Es necesario realizar una limpieza del dataset antes de proceder con el ejercicio pues contiene registros duplicados que arrojaran un problema en el numero de registros de los titulos de las peliculas. 

Para lo cual es necesario eliminar duplicados que contiene el dataset de peliculas y proceder con el ejercicio

### Paso 3: Transformando los datos

Tal y como lo hicimos en el [Ejercicio 1](https://experiencia21.tec.mx/courses/481176/assignments/15386625?module_item_id=28379086), necesitamos transformar el dataframe de `weighted_ratings` a una matriz usuario-item donde las filas representan a los usuarios y las columnas representan a las peliculas. Las celdas en esta matriz contendran el feedback implicito que en este caso es el numero de veces que un usuario ha visto una pelicula.

La funcion  `create_X()` crea una matriz de dispersion **X** con 4 diccionarios de mapeo:

- **user_mapper:** mapea user id al user index
- **item_mapper:** mapea item id al item index
- **user_inv_mapper:** mapea user index al user id
- **item_inv_mapper:** mapea item index al item id

Necesitamos estos diccionario por que hay que mapear las filas y columnas con la matriz de utilidad que les corresponde al user ID con su item ID respectivamente.

Esta matriz dispersa **usuario-item** es una matriz que se obtiene al `usar scipy.sparse.csr_matrix`que almacena los datos de una manera dispersa.

In [4]:
def create_X(df: pd.DataFrame):
    """
    Generates a sparse matrix from user_items_df dataframe.
    
    Args:
        df: pandas dataframe
    
    Returns:
        X: sparse matrix
        user_mapper: dict that maps user id's to user indices
        user_inv_mapper: dict that maps user indices to user id's
        item_mapper: dict that maps item id's to item indices
        item_inv_mapper: dict that maps item indices to item id's
    """
    N = df['user_id'].nunique()
    M = df['item_id'].nunique()

    user_mapper = dict(zip(np.unique(df["user_id"]), list(range(N))))
    item_mapper = dict(zip(np.unique(df["item_id"]), list(range(M))))
    
    user_inv_mapper = dict(zip(list(range(N)), np.unique(df["user_id"])))
    item_inv_mapper = dict(zip(list(range(M)), np.unique(df["item_id"])))
    
    user_index = [user_mapper[i] for i in df['user_id']]
    item_index = [item_mapper[i] for i in df['item_id']]

    X = csr_matrix((df["weighted_rating"], (item_index, user_index)), shape=(M, N))
    
    return X, user_mapper, item_mapper, user_inv_mapper, item_inv_mapper

In [5]:
X, user_mapper, item_mapper, user_inv_mapper, item_inv_mapper = create_X(user_items_df)

### Creando los Mapeos de los titulos de las peliculas

Necesitamos traducir el titulo de una pelicula a partir de su indice en la matriz usuario-item y vice versa. Vamos a crear dos funciones que nos ayuden con esta traduccion.

- `get_item_index()` - convierte el titulo de una pelicula a su indice. Hace uso de la funcion de comparacion de strings que se le pasan a [fuzzywuzzy](https://github.com/seatgeek/fuzzywuzzy) 
 para obtener el titulo de una pelicula que se le pase. Esto significa que no necesitamos saber la forma de escribir o el formato de una pelicula para obtener su indice.

- `get_item_item_name()` - convierte el indice de una pelicula a su titulo.

In [6]:
from fuzzywuzzy import process

def item_finder(item_name):
    all_item_names = user_items_df['item_name'].unique().tolist()
    closest_match = process.extractOne(item_name,all_item_names)
    return closest_match[0]

item_item_name_mapper = dict(zip(user_items_df['item_name'], user_items_df['item_id']))
item_item_name_inv_mapper = dict(zip(user_items_df['item_id'], user_items_df['item_name']))

def get_item_index(item_name):
    fuzzy_item_name = item_finder(item_name)
    item_id = item_item_name_mapper[fuzzy_item_name]
    item_idx = item_mapper[item_id]
    return item_idx

def get_item_item_name(item_idx): 
    item_id = item_inv_mapper[item_idx]
    item_name = item_item_name_inv_mapper[item_id]
    return item_name 

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Vamos a probar esta funcion para obtener el indice de `Jurassic Park`. 

In [7]:
get_item_index('warframe')

1769

Utilizemos el indice obtenido con la funcion `get_item_item_name()`. Tendremos que obtener el titulo de Jurassic Park.

In [8]:
get_item_item_name(1769)

'Warframe'

Con esto podemos comprobar que las funciones nos permitiran interpretar las recomendaciones obtenidas del sistema.

### Paso 4: Construyendo el modelo de modelo de Recomendacion de Feedback Implicito

Una vez que hemos transformado nuestros datos ahora si podemos empezar a construir nuestro modelo de recomendacion.


La libreria [implicit](https://github.com/benfred/implicit) esta basada en un factorizacion de matrices (tomado del algebra lineal). Esto nos permite hallar caracteristicas
latentes que se esconden en las interacciones entre los usuarios y las peliculas. Estas caracteristicas latentes nos brindan una representacion mas compacta de los gustos
de los usuarios y la descripcion de un item. La factorizacion matricial es particularmente util para datos muy dispersos y puede mejorar la calidad de las recomendaciones
obtenidas. El algoritmo opera al factorizar la matris usuario-item en dos matrices:

- matriz usuario-factorers  (n_users, k)
- matriz item-factorers     (k, n_items)

Reduciremos las dimensiones de nuestra matriz original a nuestras dimensiones particulares. No es posible interpretar cada caracteristica latente $k$. Sin embargo,
podemos suponer que una caracteristica latente puede representar a los usuarios que gusten de comedia romantica de los 90s, mientras que otra caracteristica lantente
puede representar a peliculas independientes extranjeras.


$$X_{mn} \approx P_{mk} \times Q_{nk}^T = \hat{X}$$



En el caso de una factorizacion matricial tradicional como [SVD](https://www.freecodecamp.org/news/singular-value-decomposition-vs-matrix-factorization-in-recommender-systems-b1e99bc73599/) lo que hariamos seria intentar resolver la factorizacion de una sola vez, sin embargo esto resultaria muy costoso computacionalmente. Otra forma de atacar este problem es utilizando una tecnica denominada
[Minimos Cuadrados Alternos, Alternating Least Squares (ALS)](https://sophwats.github.io/2018-04-05-gentle-als.html). Ocupando ALS, podemos resolver una matriz de factores a la vez:

- Paso 1: Fijamos la matriz de factores de usuario (user-factor) y resolvemos la matriz de factores de elementos (item-factor)
- Paso 2: Fijamos la matriz de factores de elementos (item-factor) y resolvemos la matriz de factores de usuario (user-factor)

Al alternar los pasos 1 y 2 hasta que el producto punto de la matriz de factores de elementos (item-factor) y la matriz de factores de usuarios (user-item) es aproximadamente igual a la matrix original X (user-item). Este procedimiento es comptacionalmente menos costoso y puede ser parelelizado.

La libreria `implicit` implementa una factorizacion matricial utilizando ALS (puedes consultar los detalles [aqui](https://implicit.readthedocs.io/en/latest/als.html))

In [9]:
model = implicit.als.AlternatingLeastSquares(factors=50)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 12 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


Este modelo viene con algunos hyperparametros que deben ser ajustados para generar resultados optimos:

- los factores ($k$): numero de factores latentes,
- regularizacion ($\lambda$): evita que el modelo caiga en overfitting durante el entrenamiento

Para este ejercicio definiremos $k = 50$ y $\lambda = 0.01$ como los valores a utilizar. 

El siguiente paso ahora es ajustar nuestro modelo a la matriz user-item.


In [10]:
model.fit(X.T.tocsr())

  0%|          | 0/15 [00:00<?, ?it/s]


Ahora pongamos a prueba las recomendaciones de nuestro modelo. Podemos utilizar el metodo `similar_items()` que nos muestra las peliculas mas relevantes dada una pelicula en especifico. De igual forma, podemos utilizar la funcion `get_item_index()` para obtener el indice de la pelicula si es que es una pelicula que nos interesa a partir de las recomendaciones obtenidas.

In [11]:
item_of_interest = 'Fallout : New Vegas'

item_index = get_item_index(item_of_interest)
related = model.similar_items(item_index)
related

(array([ 527,  526,  790,  789,  520, 5753,  791,  523,  522, 1326],
       dtype=int32),
 array([1.0000001 , 0.88668185, 0.77981985, 0.7647837 , 0.7171778 ,
        0.67646414, 0.67304975, 0.6524359 , 0.6365224 , 0.56436807],
       dtype=float32))

Lo que obtenemos de `similar_items()` no es facil de leer por lo que necesitamos de la funcion `get_item_item_name()` para interpretar los resultados.

In [12]:
print(f"Por que jugaste {item_finder(item_of_interest)} te pueden interesar los siguientes:")
for t, r in zip(related[0], related[1]):
    
    recommended_item_name = get_item_item_name(t)
    if recommended_item_name != item_finder(item_of_interest):
        print(recommended_item_name)



Por que jugaste Fallout: New Vegas te pueden interesar los siguientes:
Fallout 3 - Game of the Year Edition
Fallout 2
Fallout
Fallout 3
Fallout 4
Fallout Tactics
The Elder Scrolls IV: Oblivion 
The Elder Scrolls III: Morrowind
Fable - The Lost Chapters


Al usar el weighted_rating de los usuarios como feedback implicito, los resultados se ven bien. Intenta cambiando la variable `item_of_interest`.

### Paso 5: Generando las recomendaciones del usuario

Una caracteristica interesante de `implicit` es que puedes obtener recomendaciones personalizadas para un usuario determinado. Intentemos ver los resultados con un usuario especifico de nuestro dataset.

In [13]:
user_id = 'evcentric'

In [14]:
user_user_items_df = user_items_df[user_items_df['user_id']==user_id]
user_user_items_df = user_user_items_df.sort_values('weighted_rating', ascending=False)
print(f"El numero de peliculas rankeadas por el usuario {user_id} es de: {user_user_items_df['item_id'].nunique()}")

El numero de peliculas rankeadas por el usuario evcentric es de: 100


En este caso vemos que el usuario 90 miro 54 peliculas y el weighted_rating de su favoritas son:

In [15]:
user_user_items_df = user_items_df[user_items_df['user_id']==user_id]
user_user_items_df = user_user_items_df.sort_values('weighted_rating', ascending=False)
top_5 = user_user_items_df.head()
top_5

,item_id,item_name,playtime_forever,playtime_2weeks,user_id,log_ptime,exp_ptime_2week,log_ptime_scaled,exp_ptime_2week_scaled,combined_rating,recommend,weighted_rating
751,466170,Idling to Rule the Gods,28545.0,1554.0,evcentric,10.259272,10.0,3.772012,5.0,4.508805,NaN,4.508805
692,230410,Warframe,1381.0,59.0,evcentric,7.231287,10.0,2.578049,5.0,4.031220,NaN,4.031220
712,275850,No Man's Sky,1219.0,141.0,evcentric,7.106606,10.0,2.528886,5.0,4.011555,NaN,4.011555
711,211820,Starbound,67.0,67.0,evcentric,4.219508,10.0,1.390477,5.0,3.556191,NaN,3.556191
694,224500,Gnomoria,13618.0,0.0,evcentric,9.519221,5.0,3.480203,0.0,1.392081,1.0,2.784162


Las peliculas con el menor weighted_rating son:

In [16]:
bottom_5 = user_user_items_df[user_user_items_df['weighted_rating']<5].tail()
bottom_5

,item_id,item_name,playtime_forever,playtime_2weeks,user_id,log_ptime,exp_ptime_2week,log_ptime_scaled,exp_ptime_2week_scaled,combined_rating,recommend,weighted_rating
702,239350,Spelunky,5.0,0.0,evcentric,1.791759,5.0,0.433193,0.0,0.173277,NaN,0.173277
675,2290,Final DOOM,4.0,0.0,evcentric,1.609438,5.0,0.361302,0.0,0.144521,NaN,0.144521
669,67000,The Polynomial,4.0,0.0,evcentric,1.609438,5.0,0.361302,0.0,0.144521,NaN,0.144521
715,285310,RollerCoaster Tycoon: Deluxe,2.0,0.0,evcentric,1.098612,5.0,0.159879,0.0,0.063951,NaN,0.063951
725,328080,Retro-Pixel Castles,2.0,0.0,evcentric,1.098612,5.0,0.159879,0.0,0.063951,NaN,0.063951


A partir de las preferencias anteriores, podemos inferir algo acerca del usuario 90. Veamos que recomendaciones se pueden generar para este usuario en particular.

Utilizaremos `recommend()` que utiliza el indice del usuario y lo transpone con la matriz user-item.

In [17]:
X_t = X.T.tocsr()
user_idx = user_mapper[user_id]
recommendations = model.recommend(user_idx, X_t[user_idx])
recommendations

(array([1613, 6764, 2263, 2389, 1007, 1791, 1276, 4711,   19,    6],
       dtype=int32),
 array([0.5589234 , 0.4962295 , 0.4939361 , 0.4539813 , 0.44857654,
        0.3932966 , 0.3927964 , 0.38852093, 0.38602477, 0.3384616 ],
       dtype=float32))

No podemos interpretar los resultados obtenidos pues estan listados los indices. Hagamos una conversion del indice al titulo de las peliculas recomendadas.

In [18]:
for t, r in zip(recommendations[0], recommendations[1]):
    recommended_item_name = get_item_item_name(t)
    print(recommended_item_name)

Kerbal Space Program
Stardew Valley
Cities: Skylines
Borderlands: The Pre-Sequel
Saints Row: The Third
Killing Floor 2
XCOM: Enemy Unknown
ARK: Survival Evolved
Left 4 Dead
Half-Life


In [19]:
from pathos.multiprocessing import ProcessingPool as Pool
from tqdm import tqdm
import time
# 
# def process_user(user_id, numero_recom):
#     user_ra = user_items_df[user_items_df['user_id'] == user_id]
#     ground_truth = user_ra['item_id'].tolist()
#     
#     user_idx = user_mapper[user_id]
#     recommendations_s = model.recommend(user_idx, X_t[user_idx])
#     
#     return user_id, recommendations_s, ground_truth
# 
# def gen_recom_ground_truth_implicity(numero_recom, num_cores):
#     recommendations = {}
#     ground_truth = {}
#     # Fetch unique user IDs
#     unique_user_ids = user_items_df['user_id'].unique()
#     
#     # Create a Pool with the specified number of cores
#     with Pool(num_cores) as pool:
#         results = list(tqdm(pool.imap(lambda user_id: process_user(user_id, numero_recom), unique_user_ids),
#                             total=len(unique_user_ids), desc="Processing Users"))
# 
#     # Collect results from multiprocessing
#     for user_id, user_recommendations, user_ground_truth in results:
#         recommendations[user_id] = user_recommendations
#         ground_truth[user_id] = user_ground_truth
# 
#     return recommendations, ground_truth


In [20]:
#user_indices = np.arange(X_t.shape[0])
#ids, scores = model.recommend(user_indices, X_t, N=10)

In [21]:
#recommendations_dict_imp_direct = {user_inv_mapper[i]: ids[i].tolist() for i in range(len(user_indices))}
#recommendations_dict_imp_direct

In [22]:

# if __name__ == '__main__':
#     start_time = time.time()
#     numero_recom = 10
#     num_cores = 10
#     recommendations_dict_imp, ground_truth_dict_imp = gen_recom_ground_truth_implicity(numero_recom, num_cores)
#     end_time = time.time()
# 
#     print(f"Time taken: {end_time - start_time} seconds")
print("Esta sección se ejecuto anteriormente y se guardaron los resultados para evitar el tiempo")

Esta sección se ejecuto anteriormente y se guardaron los resultados para evitar el tiempo


In [23]:
import pickle
time_implicity = 9290.773648023605
# with open('recommendations_dict_imp.pkl', 'wb') as f:
#     pickle.dump(recommendations_dict_imp, f)
# 
# with open('ground_truth_dict_imp.pkl', 'wb') as f:
#     pickle.dump(ground_truth_dict_imp, f)

#Reconstrucción de datos
with open('recommendations_dict_imp.pkl', 'rb') as f:
    recommendations_dict_imp = pickle.load(f)

with open('ground_truth_dict_imp.pkl', 'rb') as f:
    ground_truth_dict_imp = pickle.load(f)

In [24]:
recommendations_imp = [recommendations_dict_imp[user][0].tolist() for user in recommendations_dict_imp]
ground_truth_imp = [ground_truth_dict_imp[user] for user in ground_truth_dict_imp]

In [25]:
from sklearn.metrics import mean_squared_error
from math import sqrt

# Extract predicted and actual ratings
predicted_ratings = []
actual_ratings = []

for user in ground_truth_dict_imp:
    if user in recommendations_dict_imp:
        predicted_items = recommendations_dict_imp[user][0]
        predicted_scores = recommendations_dict_imp[user][1]

        ground_truth_items = ground_truth_dict_imp[user]

        # Match predicted ratings with actual ratings
        for item in ground_truth_items:
            if item in predicted_items:
                index = np.where(predicted_items == item)[0][0]
                predicted_ratings.append(predicted_scores[index])
                actual_ratings.append(1)  # Assuming ground truth ratings are implicit feedback with rating = 1

# Convert to numpy arrays
predicted_ratings = np.array(predicted_ratings)
actual_ratings = np.array(actual_ratings)

# Calculate RMSE
rmse_impli = sqrt(mean_squared_error(actual_ratings, predicted_ratings))
print("RMSE:", rmse_impli)

RMSE: 0.7227867224313563


In [28]:

def truncated_precision_at_k(actual, predicted, k=10):
    if not actual:
        return 0.0
    predicted = predicted[:k]
    relevant = len(set(predicted) & set(actual))
    return relevant / min(k, len(actual))

def truncated_average_precision_at_k(actual, predicted, k=10):
    if not actual:
        return 0.0
    predicted = predicted[:k]
    score = 0.0
    num_hits = 0.0
    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)
    return score / min(k, len(actual))

def hit_at_k(actual, predicted, k=10):
    if not actual:
        return 0.0
    predicted = predicted[:k]
    return 1.0 if len(set(predicted) & set(actual)) > 0 else 0.0

def reciprocal_rank_at_k(actual, predicted, k=10):
    if not actual:
        return 0.0
    predicted = predicted[:k]
    for i, p in enumerate(predicted):
        if p in actual:
            return 1.0 / (i + 1.0)
    return 0.0
def evaluate_recommendations(ground_truth, recommendations, k=5, rmse_impli=None, time_implicity=None):
    metrics = {
        'Truncated Precision@K': np.mean([truncated_precision_at_k(a, p, k) for a, p in zip(ground_truth, recommendations)]),
        'Truncated Average Precision@K': np.mean([truncated_average_precision_at_k(a, p, k) for a, p in zip(ground_truth, recommendations)]),
        'Hit@K': np.mean([hit_at_k(a, p, k) for a, p in zip(ground_truth, recommendations)]),
        'Reciprocal Rank@K': np.mean([reciprocal_rank_at_k(a, p, k) for a, p in zip(ground_truth, recommendations)]),
        'RMSE': rmse_impli,
        'Tiempo Recomendación': time_implicity
    }
    return metrics

metrics_imp = evaluate_recommendations(ground_truth_imp, recommendations_imp, k=10)

In [29]:
metrics_df = pd.DataFrame.from_dict(metrics_imp, orient='index', columns=['Implicit']).rename_axis('Metrics')
metrics_df

,Implicit
Metrics,
Truncated Precision@K,0.002752
Truncated Average Precision@K,0.001182
Hit@K,0.021931
Reciprocal Rank@K,0.007583
RMSE,NaN
Tiempo Recomendación,NaN
